<a href="https://colab.research.google.com/github/Tony181189/JPMorgan-Challenge/blob/main/JPMorgan_04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [14]:
# 1. Carregar os dados
df = pd.read_csv('Task 3 and 4_Loan_Data.csv')

In [16]:
# 2. Preparar os dados
X = df[['credit_lines_outstanding', 'loan_amt_outstanding', 'total_debt_outstanding',
        'income', 'years_employed', 'fico_score']]
y = df['default']

# Normalizar os dados
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [17]:
# 3. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [18]:
# 4. Treinar o modelo principal
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

LogisticRegression(random_state=42)

In [19]:
# 5. Avaliar a performance do modelo principal
y_pred_proba = model.predict_proba(X_test)[:, 1]

In [20]:
# Calcular o MSE para cada variável individualmente
mse_values = {}
for i, column in enumerate(X.columns):
    # Treinar um modelo separado para cada variável
    X_train_single = X_train[:, i].reshape(-1, 1)
    X_test_single = X_test[:, i].reshape(-1, 1)

    model_single = LogisticRegression(random_state=42)
    model_single.fit(X_train_single, y_train)

    y_pred_proba_single = model_single.predict_proba(X_test_single)[:, 1]
    mse = mean_squared_error(y_test, y_pred_proba_single)
    mse_values[column] = mse

In [21]:
# Exibir os MSEs individuais
for column, mse in mse_values.items():
    print(f"Erro Quadrático Médio (MSE) para {column}: {mse:.4f}")

Erro Quadrático Médio (MSE) para credit_lines_outstanding: 0.0198
Erro Quadrático Médio (MSE) para loan_amt_outstanding: 0.1420
Erro Quadrático Médio (MSE) para total_debt_outstanding: 0.0483
Erro Quadrático Médio (MSE) para income: 0.1438
Erro Quadrático Médio (MSE) para years_employed: 0.1330
Erro Quadrático Médio (MSE) para fico_score: 0.1273


In [24]:
# Função para prever a probabilidade de inadimplência e verificar se está acima de 90%
def prever_inadimplencia():
    print("\nPreencha as informações do cliente:")

    credit_lines_outstanding = float(input("Linhas de crédito pendente: "))
    loan_amt_outstanding = float(input("Valor do empréstimo pendente: "))
    total_debt_outstanding = float(input("Dívida total pendente: "))
    income = float(input("Renda: "))
    years_employed = float(input("Anos empregado: "))
    fico_score = float(input("Score: "))

    # Criar um DataFrame com as entradas
    entrada = pd.DataFrame([[credit_lines_outstanding, loan_amt_outstanding, total_debt_outstanding,
                             income, years_employed, fico_score]],
                           columns=['credit_lines_outstanding', 'loan_amt_outstanding',
                                    'total_debt_outstanding', 'income', 'years_employed', 'fico_score'])

    # Normalizar a entrada
    entrada_scaled = scaler.transform(entrada)

    # Prever a probabilidade de inadimplência
    probabilidade = model.predict_proba(entrada_scaled)[:, 1]

    print(f"\nA probabilidade de inadimplência é: {probabilidade[0]:.2%}")

    # Verificar se a probabilidade é maior ou igual a 90%
    if probabilidade >= 0.90:
        print("Este cliente provavelmente ficará inadimplente.")
    else:
        print("Este cliente tem uma chance menor de inadimplência.")

In [25]:
# Chamar a função para prever a inadimplência
prever_inadimplencia()


Preencha as informações do cliente:
Linhas de crédito pendente: 5
Valor do empréstimo pendente: 2000
Dívida total pendente: 10000
Renda: 30000
Anos empregado: 8
Score: 400

A probabilidade de inadimplência é: 99.98%
Este cliente provavelmente ficará inadimplente.
